In [203]:
from sklearn import metrics
from os import path, listdir
from sklearn.datasets import load_svmlight_files
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib  
from sklearn import preprocessing
from sklearn import metrics
from fastFM import als,sgd,mcmc
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from scipy.sparse import csc_matrix, csr_matrix

In [204]:
def load_dataset():
    header = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
    df_user = pd.read_csv(r'/data/FM_data/u.user', sep='|', names=header)

    header = ['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
              'Animation', 'Children',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
              'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western']
    df_item = pd.read_csv(r'/data/FM_data/u.item', sep='|', names=header,
                          encoding="ISO-8859-1")
    df_item = df_item.drop(['title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown'], axis=1)

    df_user['age'] = pd.cut(df_user['age'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                            labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90',
                                    '90-100'])
    df_user = pd.get_dummies(df_user, columns=['gender', 'occupation', 'age'])
    df_user = df_user.drop(['zip_code'], axis=1)

    user_features = df_user.columns.values.tolist()
    movie_features = df_item.columns.values.tolist()
    cols = user_features + movie_features

    header = ['user_id', 'item_id', 'rating', 'timestamp']
    df_train = pd.read_csv(r'/data/FM_data/ua.base', sep='\t', names=header)
#     df_train['rating'] = df_train.rating.apply(lambda x: 1 if int(x) == 5 else 0)
    df_train = df_train.merge(df_user, on='user_id', how='left')
    df_train = df_train.merge(df_item, on='item_id', how='left')

    df_test = pd.read_csv(r'/data/FM_data/ua.test', sep='\t', names=header)
#     df_test['rating'] = df_test.rating.apply(lambda x: 1 if int(x) == 5 else 0)
    df_test = df_test.merge(df_user, on='user_id', how='left')
    df_test = df_test.merge(df_item, on='item_id', how='left')
#     train_labels = onehot_encoder(df_train['rating'].astype(np.int32), 2)
#     test_labels = onehot_encoder(df_test['rating'].astype(np.int32), 2)
    train_labels = df_train['rating']
    test_labels = df_test['rating']
    return df_train[cols].values, train_labels, df_test[cols].values, test_labels

In [205]:
# 获取样本数据
X_train, Y_train, X_test, Y_test = load_dataset()

In [206]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(90570, 53)
(90570,)
(9430, 53)
(9430,)


In [207]:
new_X_train = csc_matrix(X_train.astype(float))
new_Y_train = Y_train.as_matrix().astype(float)

new_X_test = csc_matrix(X_test.astype(float))
new_Y_test = Y_test.as_matrix().astype(float)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [208]:
new_Y_test

array([ 4.,  4.,  4., ...,  1.,  4.,  2.])

In [209]:
# from fastFM.datasets import make_user_item_regression
# from sklearn.model_selection import train_test_split
# X, y, _ = make_user_item_regression(label_stdev=.4)
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [210]:
# 3.1 建立FM模型，并且设定参数  
fm_model = mcmc.FMRegression(n_iter=1000)

In [211]:
# 3.2 训练FM模型
fm_model.fit_predict(new_X_train, new_Y_train, new_X_test)

array([ -6.39811298,  -2.32113299,  -8.99595138, ...,  13.44398612,
         0.51628555,   2.2937826 ])

In [212]:
# 3.3 采用测试集验证模型离线指标  
# 训练集AUC
probs_train= fm_model.predict(new_X_train)

mse1 = metrics.mean_squared_error(new_Y_train, probs_train)
print("Train mse: %s"%(mse1))

Train mse: 6.57563747694


In [213]:
#测试集AUC
probs_train= fm_model.predict(new_X_test)

mse1 = metrics.mean_squared_error(new_Y_test, probs_train)
print("Train mse: %s"%(mse1))

Train mse: 7.58428935587


In [214]:
# 3.4 打印模型参数
print("参数:",fm_model.get_params)   
print("=========FM模型预测的结果=========")  
print(fm_model.predict(new_X_test[0:5]))     #sigmoid函数转化的值，即：概率p  

参数: <bound method BaseEstimator.get_params of FMRegression(copy_X=True, init_stdev=0.1, n_iter=1000, random_state=123,
       rank=8)>
=========FM模型预测的结果=========
[ 4.08887938  6.4909626   4.59781272  3.12929889  8.54041709]


In [215]:
# 3.5 模型保存
joblib.dump(fm_model,"FM.model")  
#模型加载
print("模型加载")
load_FM = joblib.load("FM.model")    
print(load_FM.predict(new_X_test))

模型加载
[ 4.08887938  6.4909626   4.59781272 ...,  7.60837036  2.37327394
  3.76661323]
